[View in Colaboratory](https://colab.research.google.com/github/Dark-Sied/Word_embeddings_experiments/blob/master/H2E_classifier.ipynb)

In [4]:
from google.colab import files
files.upload()


Saving stopwords_list.txt to stopwords_list.txt


{'stopwords_list.txt': b'ke\nka\nek\nmein\nki\nhai\nyah\naur\nse\nhain\nko\npar\niss\nhota\njo\nkar\nme\ngaya\nkarne\nkiya\nliye\napne\nne\nbani\nnahi\ntoh\nhi\nya\navam\ndiya\nho\niska\ntha\ndhvara\nhua\ntak\nsaath\nkarna\nvaale\nbaad\nliya\naap\nkuchh\nsakte\nkisi\nye\niska\nsabse\nismein\nthe\ndo\nhone\nvah\nve\nkarte\nbahut\nkaha\nvarg\nkai\nkarein\nhoti\napni\nunke\nthi\nyadi\nhui\njaa\nna\nise\nkehte\nkahte\njab\nhote\nkoi\nhue\nva\nabhi\njaise\nsabhi\nkarta\nunki\ntarah\nuss\naadi\nkul\nraha\niski\nsakta\nrahe\nunka\nissi\nrakhein\napna\npe\nuske\n'}

In [86]:
!ls

english_text.csv   my_first_model.h5  stopwords_list.txt
hinglish_text.csv  sample_data


In [0]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import random
import nltk
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import load_model


from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [88]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [89]:
english = pd.read_csv("english_text.csv")
new_english = english[:300]
new_english.head()


,Unnamed: 0,text
0,0,Harry Potter and the Sorcerers Stone
1,1,CHAPTER ONE
2,2,THE BOY WHO LIVED
3,3,Mr and Mrs Dursley of number four Privet Drive...
4,4,Mr Dursley was the director of a firm called G...


In [90]:
hinglish = pd.read_csv("hinglish_text.csv")
hinglish.head()


,Unnamed: 0,text
0,0,ek din mujhe yunivarsal kampani ke karl laeeml...
1,1,jess robins nam ke ek yuva jo aisene kampani k...
2,2,us rat mainne robins ko dinar par aamantrit ki...
3,3,main sataven aasaman par tha prastav itana sha...
4,4,halanki mere shak ne sir uthana shuroo kar diy...


In [91]:
print("Length of english data = ", len(new_english))
print("Length of hinglish data = ", len(hinglish))

Length of english data =  300
Length of hinglish data =  4470


In [0]:
def remove_english_stopword(text):
  stop_words = set(stopwords.words("english"))
  words = [[w for w in s.lower().split() if w not in stop_words]for s in text]
  return words 

In [0]:
def remove_hindi_stopwords(text):
  stop_word = []
  with open("stopwords_list.txt") as f:
    word = f.readlines()
  #   print(word)
    for i in word:
  #     print(i)
      stop_word.append(i.rstrip())
    print(stop_word)
    
    word = [[w for w in s.lower().split() if w not in stop_word]for s in text]
    return word

In [0]:
eng_words = remove_english_stopword(new_english["text"])

In [95]:

print(eng_words)

[['harry', 'potter', 'sorcerers', 'stone'], ['chapter', 'one'], ['boy', 'lived'], ['mr', 'mrs', 'dursley', 'number', 'four', 'privet', 'drive', 'proud', 'say', 'perfectly', 'normal', 'thank', 'much', 'last', 'people', 'youd', 'expect', 'involved', 'anything', 'strange', 'mysterious', 'didnt', 'hold', 'nonsense'], ['mr', 'dursley', 'director', 'firm', 'called', 'grunnings', 'made', 'drills', 'big', 'beefy', 'man', 'hardly', 'neck', 'although', 'large', 'mustache', 'mrs', 'dursley', 'thin', 'blonde', 'nearly', 'twice', 'usual', 'amount', 'neck', 'came', 'useful', 'spent', 'much', 'time', 'craning', 'garden', 'fences', 'spying', 'neighbors', 'dursleys', 'small', 'son', 'called', 'dudley', 'opinion', 'finer', 'boy', 'anywhere'], ['dursleys', 'everything', 'wanted', 'also', 'secret', 'greatest', 'fear', 'somebody', 'would', 'discover', 'didnt', 'think', 'could', 'bear', 'anyone', 'found', 'potters', 'mrs', 'potter', 'mrs', 'dursleys', 'sister', 'hadnt', 'met', 'several', 'years', 'fact', 'm

In [96]:
hing_words = remove_hindi_stopwords(hinglish["text"])

['ke', 'ka', 'ek', 'mein', 'ki', 'hai', 'yah', 'aur', 'se', 'hain', 'ko', 'par', 'iss', 'hota', 'jo', 'kar', 'me', 'gaya', 'karne', 'kiya', 'liye', 'apne', 'ne', 'bani', 'nahi', 'toh', 'hi', 'ya', 'avam', 'diya', 'ho', 'iska', 'tha', 'dhvara', 'hua', 'tak', 'saath', 'karna', 'vaale', 'baad', 'liya', 'aap', 'kuchh', 'sakte', 'kisi', 'ye', 'iska', 'sabse', 'ismein', 'the', 'do', 'hone', 'vah', 've', 'karte', 'bahut', 'kaha', 'varg', 'kai', 'karein', 'hoti', 'apni', 'unke', 'thi', 'yadi', 'hui', 'jaa', 'na', 'ise', 'kehte', 'kahte', 'jab', 'hote', 'koi', 'hue', 'va', 'abhi', 'jaise', 'sabhi', 'karta', 'unki', 'tarah', 'uss', 'aadi', 'kul', 'raha', 'iski', 'sakta', 'rahe', 'unka', 'issi', 'rakhein', 'apna', 'pe', 'uske']


In [0]:
def create_words(hw, ew):
  ls = []

  count = 0
  for i in hing_words:
    for j in i:
      ls.append((list(j),1))
  ls = ls[:5000]    
  print("Total hinglish words = ",len(ls)) 

  for i in eng_words:
    for j in i:
      ls.append((list(j),0))
      count += 1   
  print("Total english words = ",count)    
  random.shuffle(ls)  
  
  return ls
    
    

In [98]:
ls = create_words(hing_words, eng_words)

Total hinglish words =  5000
Total english words =  5559


In [99]:
len(ls)
# ls[0]

10559

In [0]:
alpha = {"a":0,"b":1,"c":2,"d":3,"e":4,"f":5,"g":6,"h":7,"i":8,"j":9,"k":10,"l":11,"m":12,"n":13,"o":14,"p":15,"q":16,"r":17,"s":18,"t":19,"u":20,"v":21,"w":22,"x":23,"y":24,"z":25}
# alpha

In [0]:
words = [w[0] for w in ls]
labels = [w[1] for w in ls]

In [0]:
max_length = len(max(words, key = len))

In [0]:
train_x, test_x, train_y, test_y = train_test_split(words, labels, shuffle = True, test_size = 0.15)

In [104]:
print("Train data : ", "train_X = " ,len(train_x), "labels = " , len(train_y))
print("Test data = ", "test_X = " ,len(test_x), "labels = " , len(test_y))

Train data :  train_X =  8975 labels =  8975
Test data =  test_X =  1584 labels =  1584


In [0]:
#Function to create indexing of characters in the word

def create_indexing(data):
  final = []

  for i in data:
    new = []
    for j in i:
      new.append(alpha[j])
    final.append(new)  

  return final  

In [0]:
final_train = create_indexing(train_x)
final_test = create_indexing(test_x)

In [107]:
len(max(final_train,key = len))

17

In [0]:
def create_padding(data):
  padded_docs = []
  for i in data:
    p = np.pad(i, (0, (max_length - len(i))), mode = "constant", constant_values = 0)
    padded_docs.append(p)

  return padded_docs

In [0]:
padded_train_docs = create_padding(final_train)
padded_test_docs = create_padding(final_test)

In [110]:
padded_train_docs[0]

array([17, 14, 14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [111]:
print("Length of padded train data = ",len(padded_train_docs))
print("Length of padded test data = ",len(padded_test_docs))

Length of padded train data =  8975
Length of padded test data =  1584


In [112]:
padded_train_docs[0].shape

(17,)

In [0]:
def create_encoded_docs(pad_docs, num_of_samples):
  num_alpha = 26

  encoded_docs = np.zeros((num_of_samples, max_length, num_alpha))
  print(encoded_docs.shape)

  for nu,i in enumerate(pad_docs):
    for n, j in enumerate(i):
      encoded_docs[nu][n] = to_categorical(j, num_classes = 26)

  return encoded_docs

In [116]:
encoded_train_docs = create_encoded_docs(padded_train_docs, len(padded_train_docs))
encoded_test_docs = create_encoded_docs(padded_test_docs, len(padded_test_docs))

(8975, 17, 26)
(1584, 17, 26)


In [0]:
#Converting labels list into arrays
train_y = np.asarray(train_y)
test_y = np.asarray(test_y)

In [118]:
print("Train Labels shape = ",train_y.shape, "Test labels shape = ",test_y.shape)

Train Labels shape =  (8975,) Test labels shape =  (1584,)


In [55]:
#Creating Model

clf = Sequential()
clf.add(LSTM(8, input_shape = (17, 26)))
clf.add(Dense(1,activation = "sigmoid"))

clf.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
clf.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8)                 1120      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 1,129
Trainable params: 1,129
Non-trainable params: 0
_________________________________________________________________


In [56]:
clf.fit(encoded_train_docs, train_y, epochs = 1, batch_size = 64, validation_data = (encoded_test_docs, test_y))

Train on 8975 samples, validate on 1584 samples
Epoch 1/1
8975/8975 [==============================] - 3s 296us/step - loss: 0.6900 - acc: 0.5444 - val_loss: 0.6845 - val_acc: 0.6717


In [0]:
clf.save("my_first_model.h5")

In [0]:
model = load_model("my_first_model.h5")

In [0]:
def predict(test_str):
  test_words = word_tokenize(test_str)
  test_char = []
  for i in test_words:
    test_char.append(list(i.lower()))
    
  test_index = create_indexing(test_char)
#   print(test_index)
  
  test_pad = create_padding(test_index)
  
  test_encode = create_encoded_docs(test_pad, len(test_pad))
  
  res = model.predict_classes(test_encode)
  
  return (res)
  

In [141]:
result = predict("hermoine")
print(result)

(1, 17, 26)
[[0]]
